In [ ]:
#install.packages('lmPerm')
library(lmPerm)
#install.packages('psych')
library(psych)
#install.packages('car')
library(car)
#install.packages('ggplot2')
library(ggplot2)
#install.packages('lme4')
library(lme4)
#install.packages('mediation')
library(mediation)
#install.packages('MatchIt')
#library(MatchIt)
#install.packages('dplyr')
library(dplyr)
#install.packages('ukbtools')
library(ukbtools)
#install.packages('lavaan')
library(lavaan)
#install.packages('fsbrain')
#library(fsbrain)
#install.packages('jmuOutlier')
library('jmuOutlier')
#install.packages('lm.beta')
library(lm.beta)
#install.packages('matrixStats')
library(matrixStats)

# Full sample

In [ ]:
ADNI_thick=read.table('/dagher/dagher11/filip/OBAD/data/ADNI_thick_QC_raw.csv',sep=',', header=T)
ADNI_rid=read.table('/dagher/dagher11/filip/OBAD/data/ADNI_Filip.csv',sep=',', header=T)
ADNI_thick=merge(ADNI_thick,select(ADNI_rid, patient_id, rid))
UKBB_thick=read.table('/dagher/dagher11/filip/OBAD/data/UKBB_fullsample_excluded.csv',sep=' ', header=T, quote='"')
ADNI_full=read.table('/dagher/dagher11/filip/OBAD/data/matched_ad_full_QC.csv',sep=' ', header=T)
UKBB_full=read.table('/dagher/dagher11/filip/OBAD/data/UKBB_matched_full_QC.csv', sep=' ', header=T)
ADNI_full=merge(ADNI_thick,ADNI_full,by.x='patient_id',by.y='ID',all.y=TRUE)
UKBB_full=merge(UKBB_thick,UKBB_full,by.x='eid',by.y='ID',all.y=TRUE)

In [ ]:
write.table(ADNI_full,'/dagher/dagher11/filip/OBAD/data/ADNI_full_ct.csv',row.names=F,col.names=T, quote=F, sep=',')
write.table(UKBB_full,'/dagher/dagher11/filip/OBAD/data/UKBB_full_ct.csv',row.names=F,col.names=T, quote=T, sep=',')

In [ ]:
mean(ADNI_full$bmi_bl[ADNI_full$dx_bl=='AD'], na.rm=T)
mean(ADNI_full$bmi_bl[ADNI_full$dx_bl=='CON'], na.rm=T)
mean(UKBB_full$body_mass_index_bmi_21001.2.0[UKBB_full$Sample=='LE'], na.rm=T)
mean(UKBB_full$body_mass_index_bmi_21001.2.0[UKBB_full$Sample=='OB'], na.rm=T)

table(ADNI_full$men[ADNI_full$dx_bl=='AD'])
table(ADNI_full$men[ADNI_full$dx_bl=='CON'])
table(UKBB_full$sex_31.0.0[UKBB_full$Sample=='LE'])
table(UKBB_full$sex_31.0.0[UKBB_full$Sample=='OB'])

## Plot cortical thickness maps

In [ ]:
structs=c('lh.caudalanteriorcingulate_thickavg',
'lh.caudalmiddlefrontal_thickavg',
'lh.cuneus_thickavg',
'lh.entorhinal_thickavg',
'lh.fusiform_thickavg',
'lh.inferiorparietal_thickavg',
'lh.inferiortemporal_thickavg',
'lh.isthmuscingulate_thickavg',
'lh.lateraloccipital_thickavg',
'lh.lateralorbitofrontal_thickavg',
'lh.lingual_thickavg',
'lh.medialorbitofrontal_thickavg',
'lh.middletemporal_thickavg',
'lh.parahippocampal_thickavg',
'lh.paracentral_thickavg',
'lh.parsopercularis_thickavg',
'lh.parsorbitalis_thickavg',
'lh.parstriangularis_thickavg',
'lh.pericalcarine_thickavg',
'lh.postcentral_thickavg',
'lh.posteriorcingulate_thickavg',
'lh.precentral_thickavg',
'lh.precuneus_thickavg',
'lh.rostralanteriorcingulate_thickavg',
'lh.rostralmiddlefrontal_thickavg',
'lh.superiorfrontal_thickavg',
'lh.superiorparietal_thickavg',
'lh.superiortemporal_thickavg',
'lh.supramarginal_thickavg',
'lh.transversetemporal_thickavg',
'lh.insula_thickavg',
'rh.caudalanteriorcingulate_thickavg',
'rh.caudalmiddlefrontal_thickavg',
'rh.cuneus_thickavg',
'rh.entorhinal_thickavg',
'rh.fusiform_thickavg',
'rh.inferiorparietal_thickavg',
'rh.inferiortemporal_thickavg',
'rh.isthmuscingulate_thickavg',
'rh.lateraloccipital_thickavg',
'rh.lateralorbitofrontal_thickavg',
'rh.lingual_thickavg',
'rh.medialorbitofrontal_thickavg',
'rh.middletemporal_thickavg',
'rh.parahippocampal_thickavg',
'rh.paracentral_thickavg',
'rh.parsopercularis_thickavg',
'rh.parsorbitalis_thickavg',
'rh.parstriangularis_thickavg',
'rh.pericalcarine_thickavg',
'rh.postcentral_thickavg',
'rh.posteriorcingulate_thickavg',
'rh.precentral_thickavg',
'rh.precuneus_thickavg',
'rh.rostralanteriorcingulate_thickavg',
'rh.rostralmiddlefrontal_thickavg',
'rh.superiorfrontal_thickavg',
'rh.superiorparietal_thickavg',
'rh.superiortemporal_thickavg',
'rh.supramarginal_thickavg',
'rh.transversetemporal_thickavg',
'rh.insula_thickavg')

In [ ]:
Stats=matrix(nrow=length(grep('thickavg',colnames(ADNI_full))),ncol=5) # 
index=1

ADNI_full$Sample=factor(ADNI_full$Sample, levels=c('CON','AD'),ordered=T)

for (i in 1:length(structs)) { #
  
    model=(lm((ADNI_full[[grep(structs[i], colnames(ADNI_full))]]) ~
                factor(Sample) + Age + factor(Sex) + factor(manufacturer) + factor(mfs), data=ADNI_full))
    out=summary(model)
    coef=lm.beta(model)
      
    
    Stats[index,][1]=structs[i]
    Stats[index,][2]=coef$coefficients[2] # estimate
    Stats[index,][3]=out$coefficients[2,4] # p-value
    Stats[index,][4]=out$coefficients[2,3] # t-value
    Stats[index,][5]=out$df[2] # df
       
     
    index=index+1
}

Stats_adj_ADNI=Stats
Stats_adj_ADNI[,3]=as.numeric(p.adjust(Stats_adj_ADNI[,3], method = 'BH'))
Stats_adj_ADNI=data.frame(Stats_adj_ADNI)
colnames(Stats_adj_ADNI)=c('measure','estimate','p','t')
Stats_adj_ADNI$estimate=as.numeric(levels(Stats_adj_ADNI$estimate))[Stats_adj_ADNI$estimate]
Stats_adj_ADNI$p=as.numeric(levels(Stats_adj_ADNI$p))[Stats_adj_ADNI$p]
Stats_adj_ADNI$t=as.numeric(levels(Stats_adj_ADNI$t))[Stats_adj_ADNI$t]

## Control

In [ ]:
Stats=matrix(nrow=length(grep('thickavg',colnames(ADNI_full))),ncol=5) # 
index=1

ADNI_full$Sample=factor(ADNI_full$Sample, levels=c('CON','AD'),ordered=T)

for (i in 1:length(structs)) { #
  
    model=(lm((ADNI_full[[grep(structs[i], colnames(ADNI_full))]]) ~
                factor(Sample) + Age + factor(Sex) + factor(manufacturer) + factor(mfs), data=ADNI_full))
    out=summary(model)
    coef=lm.beta(model)
      
    
    Stats[index,][1]=structs[i]
    Stats[index,][2]=coef$coefficients[5] # estimate
    Stats[index,][3]=out$coefficients[5,4] # p-value
    Stats[index,][4]=out$coefficients[5,3] # t-value
    Stats[index,][5]=out$df[4] # df
       
     
    index=index+1
}

Stats_adj_ADNI_con=Stats
Stats_adj_ADNI_con=data.frame(Stats_adj_ADNI_con)
colnames(Stats_adj_ADNI_con)=c('measure','estimate','p','t')
Stats_adj_ADNI_con$estimate=as.numeric(levels(Stats_adj_ADNI_con$estimate))[Stats_adj_ADNI_con$estimate]
Stats_adj_ADNI_con$p=as.numeric(levels(Stats_adj_ADNI_con$p))[Stats_adj_ADNI_con$p]
Stats_adj_ADNI_con$t=as.numeric(levels(Stats_adj_ADNI_con$t))[Stats_adj_ADNI_con$t]

In [ ]:
#Stats_adj[,4][as.numeric(Stats_adj[,3])>0.05]=NA
#Stats_adj[,4][as.numeric(Stats_adj[,4])>0]=NA 

plot_list.lh=list()
plot_list.rh=list()

for (i in 1:62) {
    if (i<32) {
        plot_list.lh[[(unlist(strsplit(unlist(strsplit(as.character(Stats_adj_ADNI[i,1]),"_"))[1],'[.]'))[2])]]=as.numeric(Stats_adj_ADNI[i,2]) }
    else {
        plot_list.rh[[(unlist(strsplit(unlist(strsplit(as.character(Stats_adj_ADNI[i,1]),"_"))[1],'[.]'))[2])]]=as.numeric(Stats_adj_ADNI[i,2])
    }
}
plot_list.lh[['bankssts']]=plot_list.lh[['middletemporal']]
plot_list.rh[['bankssts']]=plot_list.rh[['middletemporal']]

In [ ]:
atlas = 'aparc';
template_subject = 'fsaverage';
template_subjects_dir='/dagher/dagher10/yyau/toolbox/freesurfer/subjects/'
makecmap_options = list()
colFn_many_blues = colorRampPalette(RColorBrewer::brewer.pal(9, name="Blues"));
colFn_diverging= function(n) { grDevices::hcl.colors(n,palette='Blue-Red 3'); }
makecmap_options = list('colFn'=colFn_diverging, 'symm'=TRUE);


rgloptions = list("windowRect"=c(50, 50, 1000, 1000));
rglactions = list("snapshot_png"="/dagher/dagher11/filip/OBAD/figures/CT_ad_full.png")


vis.region.values.on.subject(template_subjects_dir, template_subject, atlas, 
                             plot_list.lh, 
                             plot_list.rh,
                             draw_colorbar = TRUE,
                             #surface='inflated',
                             rgloptions=rgloptions,
                             makecmap_options=makecmap_options,
                             rglactions=rglactions, bg='curv');

## UKBB

In [ ]:
UKBB_full$date_diff=as.numeric(as.Date(UKBB_full$date_of_attending_assessment_centre_53.2.0)-as.Date('2014-04-30'))
UKBB_full$date_diff_sq=(as.numeric(UKBB_full$date_diff))^2


In [ ]:
Stats=matrix(nrow=length(grep('thickness',colnames(UKBB_full))),ncol=5) # 
index=1

UKBB_full$Sample=factor(UKBB_full$Sample, levels=c('LE','OB'),ordered=T)


for (i in grep('thickness',colnames(UKBB_full))) { #
  
    model=(lm((UKBB_full[[i]]) ~
                factor(Sample) + Age + factor(Sex) + factor(uk_biobank_assessment_centre_54.2.0), data=UKBB_full))
    out=summary(model)
    coef=lm.beta(model)
      
    
    Stats[index,][1]=colnames(UKBB_full)[i]
    Stats[index,][2]=coef$coefficients[2] # estimate
    Stats[index,][3]=out$coefficients[2,4] # p-value
    Stats[index,][4]=out$coefficients[2,3] # t-value
    Stats[index,][5]=out$df[2] # df
     
    index=index+1
}

Stats_adj_UKBB=Stats
Stats_adj_UKBB[,3]=as.numeric(p.adjust(Stats_adj_UKBB[,3], method = 'BH'))
Stats_adj_UKBB=data.frame(Stats_adj_UKBB)
colnames(Stats_adj_UKBB)=c('measure','estimate','p','t')
Stats_adj_UKBB$estimate=as.numeric(levels(Stats_adj_UKBB$estimate))[Stats_adj_UKBB$estimate]
Stats_adj_UKBB$p=as.numeric(levels(Stats_adj_UKBB$p))[Stats_adj_UKBB$p]
Stats_adj_UKBB$t=as.numeric(levels(Stats_adj_UKBB$t))[Stats_adj_UKBB$t]

# Control

In [ ]:
Stats=matrix(nrow=length(grep('thickness',colnames(UKBB_full))),ncol=5) # 
index=1

UKBB_full$Sample=factor(UKBB_full$Sample, levels=c('LE','OB'),ordered=T)


for (i in grep('thickness',colnames(UKBB_full))) { #
  
    model=(lm((UKBB_full[[i]]) ~
                factor(Sample) + Age + factor(Sex) + factor(uk_biobank_assessment_centre_54.2.0), data=UKBB_full))
    out=summary(model)
    coef=lm.beta(model)
      
    
    Stats[index,][1]=colnames(UKBB_full)[i]
    Stats[index,][2]=coef$coefficients[5] # estimate
    Stats[index,][3]=out$coefficients[5,4] # p-value
    Stats[index,][4]=out$coefficients[5,3] # t-value
    Stats[index,][5]=out$df[4] # df
     
    index=index+1
}

Stats_adj_UKBB_con=Stats
Stats_adj_UKBB_con=data.frame(Stats_adj_UKBB_con)
colnames(Stats_adj_UKBB_con)=c('measure','estimate','p','t')
Stats_adj_UKBB_con$estimate=as.numeric(levels(Stats_adj_UKBB_con$estimate))[Stats_adj_UKBB_con$estimate]
Stats_adj_UKBB_con$p=as.numeric(levels(Stats_adj_UKBB_con$p))[Stats_adj_UKBB_con$p]
Stats_adj_UKBB_con$t=as.numeric(levels(Stats_adj_UKBB_con$t))[Stats_adj_UKBB_con$t]

In [ ]:
#Stats_adj_UKBB[,4][as.numeric(Stats_adj_UKBB[,3])>0.05]=NA
#Stats_adj[,4][as.numeric(Stats_adj[,4])>0]=NA 

plot_list.lh=list()
plot_list.rh=list()

for (i in 1:62) {
    if (i<32) {
        plot_list.lh[[(unlist(strsplit(as.character(Stats_adj_UKBB[i,1]),"_")))[4]]]=as.numeric(Stats_adj_UKBB[i,2]) }
    else {
        plot_list.rh[[(unlist(strsplit(as.character(Stats_adj_UKBB[i,1]),"_")))[4]]]=as.numeric(Stats_adj_UKBB[i,2])
    }
}

plot_list.lh[['bankssts']]=plot_list.lh[['middletemporal']]
plot_list.rh[['bankssts']]=plot_list.rh[['middletemporal']]

In [ ]:
atlas = 'aparc';
template_subject = 'fsaverage';
template_subjects_dir='/home/bic/fmorys/.local/share/fsbrain/subjects_dir/'
makecmap_options = list()
colFn_many_blues = colorRampPalette(RColorBrewer::brewer.pal(9, name="Blues"));
colFn_diverging= function(n) { grDevices::hcl.colors(n,palette='Blue-Red 3'); }
makecmap_options = list('colFn'=colFn_diverging, 'symm'=TRUE);


rgloptions = list("windowRect"=c(50, 50, 1000, 1000));
rglactions = list("snapshot_png"="/dagher/dagher11/filip/OBAD/figures/CT_ob_full.png")


vis.region.values.on.subject(template_subjects_dir, template_subject, atlas, 
                             plot_list.lh, 
                             plot_list.rh,
                             draw_colorbar = TRUE,
                             bg='curv',
                             rgloptions=rgloptions,
                             makecmap_options=makecmap_options,
                             rglactions=rglactions);

## Tau and amyloid maps

https://neurovault.org/collections/12296/

In [ ]:
tau=c(1.13974106e+00, 3.88694712e+00,
       2.39511371e+00, 5.53503271e+00, 5.38190645e+00, 5.62174550e+00,
       6.46472202e+00, 3.82113160e+00, 4.09139808e+00, 2.02190950e+00,
       2.86058562e+00, 1.33826878e+00, 5.88404469e+00, 3.96643209e+00,
       1.09396560e+00, 2.58822830e+00, 1.62682463e+00, 2.18356348e+00,
       1.34253965e+00, 1.87654946e+00, 2.79856270e+00, 2.31431810e+00,
       3.85138689e+00, 1.36745723e+00, 3.08617677e+00, 2.58048517e+00,
       3.78359833e+00, 3.70438113e+00, 4.25749330e+00, 2.08060858e+00,
       1.83264941e+00, 1.32151448e+00, 3.60782540e+00, 2.64397057e+00,
       5.10105331e+00, 5.35011769e+00, 5.29500462e+00, 6.26608248e+00,
       3.90458355e+00, 4.18040327e+00, 1.95614101e+00, 2.74218305e+00,
       1.50811528e+00, 5.75832020e+00, 4.39307546e+00, 1.28910970e+00,
       2.47746555e+00, 1.60182662e+00, 2.08189763e+00, 1.79592073e+00,
       1.84113443e+00, 2.79200414e+00, 2.06274244e+00, 4.13677223e+00,
       1.05959101e+00, 2.58825114e+00, 2.38333756e+00, 3.17815187e+00,
       3.42950200e+00, 4.04442488e+00, 1.62788138e+00, 1.81627956e+00)

In [ ]:
amyloid=c(0.42307589, 0.43821604,0.27832654,0.13792826,0.34242532,0.4655474,0.43592951,
0.32543542,0.40300421,0.30428422,0.4261153,0.24937009,0.4867927,0.44308223,0.34912991,
0.2063827,0.42481983,0.44571424,0.43345765,0.31010989,0.31120999,0.49096764,0.28417289,
0.5524801,0.48409009,0.53447225,0.46038992,0.40673786,0.37977172,0.43856865,0.34580301,
0.44025162,0.43777316,0.27532236,0.13209678,0.33495502,0.47979976,0.43046158,0.33467261,
0.40357215,0.31724805,0.42111977,0.2464843,0.50456126,0.4458489,0.34595968,0.20939865,
0.42978596,0.46389543,0.44211144,0.3286671,0.31127205,0.48978672,0.28914124,0.53541061,
0.4784875,0.54270811,0.46145932,0.3951592,0.37729085,0.42214311,0.32533372)

## Calculate correlations between maps

In [ ]:
UKBB_est=Stats_adj_UKBB$est
UKBB_est_con=Stats_adj_UKBB_con$estimate

In [ ]:
ADNI_est=Stats_adj_ADNI$est
ADNI_est_con=Stats_adj_ADNI_con$estimate

In [ ]:
cor.test(ADNI_est, amyloid, method='spearman')
cor.test(UKBB_est, amyloid, method='spearman')
cor.test(ADNI_est, tau, method='spearman')
cor.test(UKBB_est, tau, method='spearman')
cor.test(amyloid, tau, method='spearman')

In [ ]:
cor.test(UKBB_est,ADNI_est, method='spearman')

In [ ]:
cor.test(UKBB_est_con,ADNI_est_con, method='spearman')

## Plot difference maps

In [ ]:
Stats_adj_diffplot=Stats_adj_ADNI
Stats_adj_diffplot$estimate=abs((rank(Stats_adj_ADNI$estimate))-(rank(Stats_adj_UKBB$estimate)))


In [ ]:
print(Stats_adj_diffplot[order(Stats_adj_diffplot$estimate),])

In [ ]:
#Stats_adj[,4][as.numeric(Stats_adj[,3])>0.05]=NA
#Stats_adj[,4][as.numeric(Stats_adj[,4])>0]=NA 

plot_list.lh=list()
plot_list.rh=list()

for (i in 1:62) {
    if (i<32) {
        plot_list.lh[[(unlist(strsplit(unlist(strsplit(as.character(Stats_adj_diffplot[i,1]),"_"))[1],'[.]'))[2])]]=as.numeric(Stats_adj_diffplot[i,2]) }
    else {
        plot_list.rh[[(unlist(strsplit(unlist(strsplit(as.character(Stats_adj_diffplot[i,1]),"_"))[1],'[.]'))[2])]]=as.numeric(Stats_adj_diffplot[i,2])
    }
}
plot_list.lh[['bankssts']]=plot_list.lh[['middletemporal']]
plot_list.rh[['bankssts']]=plot_list.rh[['middletemporal']]

In [ ]:
atlas = 'aparc';
template_subject = 'fsaverage';
template_subjects_dir='/dagher/dagher10/yyau/toolbox/freesurfer/subjects/'
makecmap_options = list()
colFn_many_blues = colorRampPalette(RColorBrewer::brewer.pal(9, name="Blues"));
colFn_diverging= function(n) { grDevices::hcl.colors(n,palette='YlOrRd'); }
makecmap_options = list('colFn'=colFn_diverging, 'symm'=FALSE);


rgloptions = list("windowRect"=c(50, 50, 1000, 1000));
rglactions = list("snapshot_png"="/dagher/dagher11/filip/OBAD/figures/CT_diffrank_map_full.png")


vis.region.values.on.subject(template_subjects_dir, template_subject, atlas, 
                             plot_list.lh, 
                             plot_list.rh,
                             draw_colorbar = TRUE,
                             #surface='inflated',
                             rgloptions=rgloptions,
                             makecmap_options=makecmap_options,
                             rglactions=rglactions, bg='curv');

# Obese sample

In [ ]:
ADNI_ob=read.table('/dagher/dagher11/filip/OBAD/data/matched_adob.csv',sep=' ', header=T)
UKBB_ob=read.table('/dagher/dagher11/filip/OBAD/data/UKBB_matched_adob.csv', sep=' ', header=T)
ADNI_ob=merge(ADNI_thick,ADNI_ob,by.x='patient_id',by.y='ID',all.y=TRUE)
UKBB_ob=merge(UKBB_thick,UKBB_ob,by.x='eid',by.y='ID',all.y=TRUE)

In [ ]:
mean(ADNI_ob$bmi_bl[ADNI_ob$dx_bl=='AD'], na.rm=T)
mean(ADNI_ob$bmi_bl[ADNI_ob$dx_bl=='CON'], na.rm=T)
mean(UKBB_ob$body_mass_index_bmi_21001.2.0[UKBB_ob$Sample=='LE'], na.rm=T)
mean(UKBB_ob$body_mass_index_bmi_21001.2.0[UKBB_ob$Sample=='OB'], na.rm=T)

table(ADNI_ob$men[ADNI_ob$dx_bl=='AD'])
table(ADNI_ob$men[ADNI_ob$dx_bl=='CON'])
table(UKBB_ob$sex_31.0.0[UKBB_ob$Sample=='LE'])
table(UKBB_ob$sex_31.0.0[UKBB_ob$Sample=='OB'])

## Plot cortical thickness maps

In [ ]:
Stats=matrix(nrow=length(grep('thickavg',colnames(ADNI_ob))),ncol=5) # 
index=1

ADNI_ob$Sample=factor(ADNI_ob$Sample, levels=c('CONOB','ADOB'),ordered=T)

for (i in 1:length(structs)) { #
  
    model=(lm((ADNI_ob[[grep(structs[i], colnames(ADNI_ob))]]) ~
                factor(Sample) + Age + factor(Sex) + factor(manufacturer) + factor(mfs), data=ADNI_ob))
    out=summary(model)
    coef=lm.beta(model)
      
    
    Stats[index,][1]=structs[i]
    Stats[index,][2]=coef$coefficients[2] # estimate
    Stats[index,][3]=out$coefficients[2,4] # p-value
    Stats[index,][4]=out$coefficients[2,3] # t-value
    Stats[index,][5]=out$df[2] # df
       
     
    index=index+1
}

Stats_adj_ADNI_ob=Stats
Stats_adj_ADNI_ob[,3]=as.numeric(p.adjust(Stats_adj_ADNI_ob[,3], method = 'BH'))
Stats_adj_ADNI_ob=data.frame(Stats_adj_ADNI_ob)
colnames(Stats_adj_ADNI_ob)=c('measure','estimate','p','t')
Stats_adj_ADNI_ob$estimate=as.numeric(levels(Stats_adj_ADNI_ob$estimate))[Stats_adj_ADNI_ob$estimate]
Stats_adj_ADNI_ob$p=as.numeric(levels(Stats_adj_ADNI_ob$p))[Stats_adj_ADNI_ob$p]
Stats_adj_ADNI_ob$t=as.numeric(levels(Stats_adj_ADNI_ob$t))[Stats_adj_ADNI_ob$t]

# Control

In [ ]:
Stats=matrix(nrow=length(grep('thickavg',colnames(ADNI_ob))),ncol=5) # 
index=1

ADNI_ob$Sample=factor(ADNI_ob$Sample, levels=c('CONOB','ADOB'),ordered=T)

for (i in 1:length(structs)) { #
  
    model=(lm((ADNI_ob[[grep(structs[i], colnames(ADNI_ob))]]) ~
                factor(Sample) + Age + factor(Sex) + factor(manufacturer) + factor(mfs), data=ADNI_ob))
    out=summary(model)
    coef=lm.beta(model)
    
    Stats[index,][1]=structs[i]
    Stats[index,][2]=coef$coefficients[7] # estimate
    Stats[index,][3]=out$coefficients[7,4] # p-value
    Stats[index,][4]=out$coefficients[7,3] # t-value
    Stats[index,][5]=out$df[4] # df
       
     
    index=index+1
}

Stats_adj_ADNI_ob_con=Stats
Stats_adj_ADNI_ob_con=data.frame(Stats_adj_ADNI_ob_con)
colnames(Stats_adj_ADNI_ob_con)=c('measure','estimate','p','t')
Stats_adj_ADNI_ob_con$estimate=as.numeric(levels(Stats_adj_ADNI_ob_con$estimate))[Stats_adj_ADNI_ob_con$estimate]
Stats_adj_ADNI_ob_con$p=as.numeric(levels(Stats_adj_ADNI_ob_con$p))[Stats_adj_ADNI_ob_con$p]
Stats_adj_ADNI_ob_con$t=as.numeric(levels(Stats_adj_ADNI_ob_con$t))[Stats_adj_ADNI_ob_con$t]

In [ ]:
#Stats_adj[,4][as.numeric(Stats_adj[,3])>0.05]=NA
#Stats_adj[,4][as.numeric(Stats_adj[,4])>0]=NA 

plot_list.lh=list()
plot_list.rh=list()

for (i in 1:62) {
    if (i<32) {
        plot_list.lh[[(unlist(strsplit(unlist(strsplit(as.character(Stats_adj_ADNI_ob[i,1]),"_"))[1],'[.]'))[2])]]=as.numeric(Stats_adj_ADNI_ob[i,2]) }
    else {
        plot_list.rh[[(unlist(strsplit(unlist(strsplit(as.character(Stats_adj_ADNI_ob[i,1]),"_"))[1],'[.]'))[2])]]=as.numeric(Stats_adj_ADNI_ob[i,2])
    }
}

plot_list.lh[['bankssts']]=plot_list.lh[['middletemporal']]
plot_list.rh[['bankssts']]=plot_list.rh[['middletemporal']]

In [ ]:
atlas = 'aparc';
template_subject = 'fsaverage';
template_subjects_dir='/home/bic/fmorys/.local/share/fsbrain/subjects_dir/'
makecmap_options = list()
colFn_many_blues = colorRampPalette(RColorBrewer::brewer.pal(9, name="Blues"));
colFn_diverging= function(n) { grDevices::hcl.colors(n,palette='Blue-Red 3'); }
makecmap_options = list('colFn'=colFn_diverging, 'symm'=TRUE);


rgloptions = list("windowRect"=c(50, 50, 1000, 1000));

rglactions = list("snapshot_png"="/dagher/dagher11/filip/OBAD/figures/CT_ad_obese.png")


vis.region.values.on.subject(template_subjects_dir, template_subject, atlas, 
                             plot_list.lh, 
                             plot_list.rh,
                             draw_colorbar = TRUE,
                             bg='curv',
                             rgloptions=rgloptions,
                             makecmap_options=makecmap_options,
                             rglactions=rglactions);

## UKBB

In [ ]:
Stats=matrix(nrow=length(grep('thickness',colnames(UKBB_ob))),ncol=5) # 
index=1

UKBB_ob$Sample=factor(UKBB_ob$Sample, levels=c('LE','OB'),ordered=T)


for (i in grep('thickness',colnames(UKBB_ob))) { #
  
    model=(lm((UKBB_ob[[i]]) ~
                factor(Sample) + Age + factor(Sex) + factor(uk_biobank_assessment_centre_54.2.0), data=UKBB_ob))
    out=summary(model)
    coef=lm.beta(model)
      
    
    Stats[index,][1]=colnames(UKBB_ob)[i]
    Stats[index,][2]=coef$coefficients[2] # estimate
    Stats[index,][3]=out$coefficients[2,4] # p-value
    Stats[index,][4]=out$coefficients[2,3] # t-value
    Stats[index,][5]=out$df[2] # df
     
    index=index+1
}

Stats_adj_UKBB_ob=Stats
Stats_adj_UKBB_ob[,3]=as.numeric(p.adjust(Stats_adj_UKBB_ob[,3], method = 'BH'))
Stats_adj_UKBB_ob=data.frame(Stats_adj_UKBB_ob)
colnames(Stats_adj_UKBB_ob)=c('measure','estimate','p','t')
Stats_adj_UKBB_ob$estimate=as.numeric(levels(Stats_adj_UKBB_ob$estimate))[Stats_adj_UKBB_ob$estimate]
Stats_adj_UKBB_ob$p=as.numeric(levels(Stats_adj_UKBB_ob$p))[Stats_adj_UKBB_ob$p]
Stats_adj_UKBB_ob$t=as.numeric(levels(Stats_adj_UKBB_ob$t))[Stats_adj_UKBB_ob$t]

# Control

In [ ]:
Stats=matrix(nrow=length(grep('thickness',colnames(UKBB_ob))),ncol=5) # 
index=1

UKBB_ob$Sample=factor(UKBB_ob$Sample, levels=c('LE','OB'),ordered=T)


for (i in grep('thickness',colnames(UKBB_ob))) { #
  
    model=(lm((UKBB_ob[[i]]) ~
                factor(Sample) + Age + factor(Sex) + factor(uk_biobank_assessment_centre_54.2.0), data=UKBB_ob))
    out=summary(model)
    coef=lm.beta(model)
      
    
    Stats[index,][1]=colnames(UKBB_ob)[i]
    Stats[index,][2]=coef$coefficients[5] # estimate
    Stats[index,][3]=out$coefficients[5,4] # p-value
    Stats[index,][4]=out$coefficients[5,3] # t-value
    Stats[index,][5]=out$df[4] # df
     
    index=index+1
}

Stats_adj_UKBB_ob_con=Stats
Stats_adj_UKBB_ob_con=data.frame(Stats_adj_UKBB_ob_con)
colnames(Stats_adj_UKBB_ob_con)=c('measure','estimate','p','t')
Stats_adj_UKBB_ob_con$estimate=as.numeric(levels(Stats_adj_UKBB_ob_con$estimate))[Stats_adj_UKBB_ob_con$estimate]
Stats_adj_UKBB_ob_con$p=as.numeric(levels(Stats_adj_UKBB_ob_con$p))[Stats_adj_UKBB_ob_con$p]
Stats_adj_UKBB_ob_con$t=as.numeric(levels(Stats_adj_UKBB_ob_con$t))[Stats_adj_UKBB_ob_con$t]

In [ ]:
#Stats_adj_UKBB[,4][as.numeric(Stats_adj_UKBB[,3])>0.05]=NA
#Stats_adj[,4][as.numeric(Stats_adj[,4])>0]=NA 

plot_list.lh=list()
plot_list.rh=list()

for (i in 1:62) {
    if (i<32) {
        plot_list.lh[[(unlist(strsplit(as.character(Stats_adj_UKBB_ob[i,1]),"_")))[4]]]=as.numeric(Stats_adj_UKBB_ob[i,2]) }
    else {
        plot_list.rh[[(unlist(strsplit(as.character(Stats_adj_UKBB_ob[i,1]),"_")))[4]]]=as.numeric(Stats_adj_UKBB_ob[i,2])
    }
}

plot_list.lh[['bankssts']]=plot_list.lh[['middletemporal']]
plot_list.rh[['bankssts']]=plot_list.rh[['middletemporal']]

In [ ]:
atlas = 'aparc';
template_subject = 'fsaverage';
template_subjects_dir='/home/bic/fmorys/.local/share/fsbrain/subjects_dir/'
makecmap_options = list()
colFn_many_blues = colorRampPalette(RColorBrewer::brewer.pal(9, name="Blues"));
colFn_diverging= function(n) { grDevices::hcl.colors(n,palette='Blue-Red 3'); }
makecmap_options = list('colFn'=colFn_diverging, 'symm'=TRUE);


rgloptions = list("windowRect"=c(50, 50, 1000, 1000));
rglactions = list("snapshot_png"="/dagher/dagher11/filip/OBAD/figures/CT_ob_obese.png")


vis.region.values.on.subject(template_subjects_dir, template_subject, atlas, 
                             plot_list.lh, 
                             plot_list.rh,
                             draw_colorbar = TRUE,
                             bg='curv',
                             rgloptions=rgloptions,
                             makecmap_options=makecmap_options,
                             rglactions=rglactions);

## Calculate correlations between maps

In [ ]:
UKBB_est_ob=Stats_adj_UKBB_ob$estimate
UKBB_est_ob_con=Stats_adj_UKBB_ob_con$estimate

In [ ]:
ADNI_est_ob=Stats_adj_ADNI_ob$estimate
ADNI_est_ob_con=Stats_adj_ADNI_ob_con$estimate

In [ ]:
cor.test(UKBB_est_ob,ADNI_est_ob, method='spearman')

In [ ]:
cor.test(UKBB_est_ob_con,ADNI_est_ob_con, method='spearman')

# Lean sample

In [ ]:
ADNI_le=read.table('/dagher/dagher11/filip/OBAD/data/matched_adle.csv',sep=' ', header=T)
UKBB_le=read.table('/dagher/dagher11/filip/OBAD/data/UKBB_matched_adle.csv', sep=' ', header=T)
ADNI_le=merge(ADNI_thick,ADNI_le,by.x='patient_id',by.y='ID',all.y=TRUE)
UKBB_le=merge(UKBB_thick,UKBB_le,by.x='eid',by.y='ID',all.y=TRUE)

In [ ]:
mean(ADNI_le$bmi_bl[ADNI_le$dx_bl=='AD'], na.rm=T)
mean(ADNI_le$bmi_bl[ADNI_le$dx_bl=='CON'], na.rm=T)
mean(UKBB_le$body_mass_index_bmi_21001.2.0[UKBB_le$Sample=='LE'], na.rm=T)
mean(UKBB_le$body_mass_index_bmi_21001.2.0[UKBB_le$Sample=='OB'], na.rm=T)

table(ADNI_le$men[ADNI_le$dx_bl=='AD'])
table(ADNI_le$men[ADNI_le$dx_bl=='CON'])
table(UKBB_le$sex_31.0.0[UKBB_le$Sample=='LE'])
table(UKBB_le$sex_31.0.0[UKBB_le$Sample=='OB'])

## Plot cortical thickness maps

In [ ]:
Stats=matrix(nrow=length(grep('thickavg',colnames(ADNI_le))),ncol=5) # 
index=1

ADNI_le$Sample=factor(ADNI_le$Sample, levels=c('CONLE','ADLE'),ordered=T)

for (i in 1:length(structs)) { #
  
    model=(lm((ADNI_le[[grep(structs[i], colnames(ADNI_le))]]) ~
                factor(Sample) + Age + factor(Sex) + factor(manufacturer) + factor(mfs), data=ADNI_le))
    out=summary(model)
    coef=lm.beta(model)
      
    
    Stats[index,][1]=structs[i]
    Stats[index,][2]=coef$coefficients[2] # estimate
    Stats[index,][3]=out$coefficients[2,4] # p-value
    Stats[index,][4]=out$coefficients[2,3] # t-value
    Stats[index,][5]=out$df[2] # df
       
     
    index=index+1
}

Stats_adj_ADNI_le=Stats
Stats_adj_ADNI_le[,3]=as.numeric(p.adjust(Stats_adj_ADNI_le[,3], method = 'BH'))
Stats_adj_ADNI_le=data.frame(Stats_adj_ADNI_le)
colnames(Stats_adj_ADNI_le)=c('measure','estimate','p','t')
Stats_adj_ADNI_le$estimate=as.numeric(levels(Stats_adj_ADNI_le$estimate))[Stats_adj_ADNI_le$estimate]
Stats_adj_ADNI_le$p=as.numeric(levels(Stats_adj_ADNI_le$p))[Stats_adj_ADNI_le$p]
Stats_adj_ADNI_le$t=as.numeric(levels(Stats_adj_ADNI_le$t))[Stats_adj_ADNI_le$t]

# Control

In [ ]:
Stats=matrix(nrow=length(grep('thickavg',colnames(ADNI_le))),ncol=5) # 
index=1

ADNI_le$Sample=factor(ADNI_le$Sample, levels=c('CONLE','ADLE'),ordered=T)
  
    for (i in 1:length(structs)) { #
  
    model=(lm((ADNI_le[[grep(structs[i], colnames(ADNI_le))]]) ~
                factor(Sample) + Age + factor(Sex) + factor(manufacturer) + factor(mfs), data=ADNI_le))
    out=summary(model)
    coef=lm.beta(model)
      
      
    
    Stats[index,][1]=colnames(ADNI_le)[i]
    Stats[index,][2]=coef$coefficients[7] # estimate
    Stats[index,][3]=out$coefficients[7,4] # p-value
    Stats[index,][4]=out$coefficients[7,3] # t-value
    Stats[index,][5]=out$df[4] # df
       
     
    index=index+1
}

Stats_adj_ADNI_le_con=Stats
Stats_adj_ADNI_le_con=data.frame(Stats_adj_ADNI_le_con)
colnames(Stats_adj_ADNI_le_con)=c('measure','estimate','p','t')
Stats_adj_ADNI_le_con$estimate=as.numeric(levels(Stats_adj_ADNI_le_con$estimate))[Stats_adj_ADNI_le_con$estimate]
Stats_adj_ADNI_le_con$p=as.numeric(levels(Stats_adj_ADNI_le_con$p))[Stats_adj_ADNI_le_con$p]
Stats_adj_ADNI_le_con$t=as.numeric(levels(Stats_adj_ADNI_le_con$t))[Stats_adj_ADNI_le_con$t]

In [ ]:
#Stats_adj[,4][as.numeric(Stats_adj[,3])>0.05]=NA
#Stats_adj[,4][as.numeric(Stats_adj[,4])>0]=NA 

plot_list.lh=list()
plot_list.rh=list()

for (i in 1:62) {
    if (i<32) {
        plot_list.lh[[(unlist(strsplit(unlist(strsplit(as.character(Stats_adj_ADNI_le[i,1]),"_"))[1],'[.]'))[2])]]=as.numeric(Stats_adj_ADNI_le[i,2]) }
    else {
        plot_list.rh[[(unlist(strsplit(unlist(strsplit(as.character(Stats_adj_ADNI_le[i,1]),"_"))[1],'[.]'))[2])]]=as.numeric(Stats_adj_ADNI_le[i,2])
    }
}

plot_list.lh[['bankssts']]=plot_list.lh[['middletemporal']]
plot_list.rh[['bankssts']]=plot_list.rh[['middletemporal']]

In [ ]:
atlas = 'aparc';
template_subject = 'fsaverage';
template_subjects_dir='/home/bic/fmorys/.local/share/fsbrain/subjects_dir/'
makecmap_options = list()
colFn_many_blues = colorRampPalette(RColorBrewer::brewer.pal(9, name="Blues"));
colFn_diverging= function(n) { grDevices::hcl.colors(n,palette='Blue-Red 3'); }
makecmap_options = list('colFn'=colFn_diverging, 'symm'=TRUE);


rgloptions = list("windowRect"=c(50, 50, 1000, 1000));
rglactions = list("snapshot_png"="/dagher/dagher11/filip/OBAD/figures/CT_ad_lean.png")


vis.region.values.on.subject(template_subjects_dir, template_subject, atlas, 
                             plot_list.lh, 
                             plot_list.rh,
                             draw_colorbar = TRUE,
                             bg='curv',
                             rgloptions=rgloptions,
                             makecmap_options=makecmap_options,
                             rglactions=rglactions);

## UKBB

In [ ]:
Stats=matrix(nrow=length(grep('thickness',colnames(UKBB_le))),ncol=5) # 
index=1

UKBB_le$Sample=factor(UKBB_le$Sample, levels=c('LE','OB'),ordered=T)


for (i in grep('thickness',colnames(UKBB_le))) { #
  
    model=(lm((UKBB_le[[i]]) ~
                factor(Sample) + Age + factor(Sex) + factor(uk_biobank_assessment_centre_54.2.0), data=UKBB_le))
    out=summary(model)
    coef=lm.beta(model)
      
    
    Stats[index,][1]=colnames(UKBB_le)[i]
    Stats[index,][2]=coef$coefficients[2] # estimate
    Stats[index,][3]=out$coefficients[2,4] # p-value
    Stats[index,][4]=out$coefficients[2,3] # t-value
    Stats[index,][5]=out$df[2] # df
     
    index=index+1
}

Stats_adj_UKBB_le=Stats
Stats_adj_UKBB_le[,3]=as.numeric(p.adjust(Stats_adj_UKBB_le[,3], method = 'BH'))
Stats_adj_UKBB_le=data.frame(Stats_adj_UKBB_le)
colnames(Stats_adj_UKBB_le)=c('measure','estimate','p','t')
Stats_adj_UKBB_le$estimate=as.numeric(levels(Stats_adj_UKBB_le$estimate))[Stats_adj_UKBB_le$estimate]
Stats_adj_UKBB_le$p=as.numeric(levels(Stats_adj_UKBB_le$p))[Stats_adj_UKBB_le$p]
Stats_adj_UKBB_le$t=as.numeric(levels(Stats_adj_UKBB_le$t))[Stats_adj_UKBB_le$t]

# Control

In [ ]:
Stats=matrix(nrow=length(grep('thickness',colnames(UKBB_le))),ncol=5) # 
index=1

UKBB_le$Sample=factor(UKBB_le$Sample, levels=c('LE','OB'),ordered=T)


for (i in grep('thickness',colnames(UKBB_le))) { #
  
    model=(lm((UKBB_le[[i]]) ~
                factor(Sample) + Age + factor(Sex) + factor(uk_biobank_assessment_centre_54.2.0), data=UKBB_le))
    out=summary(model)
    coef=lm.beta(model)
      
    
    Stats[index,][1]=colnames(UKBB_le)[i]
    Stats[index,][2]=coef$coefficients[5] # estimate
    Stats[index,][3]=out$coefficients[5,4] # p-value
    Stats[index,][4]=out$coefficients[5,3] # t-value
    Stats[index,][5]=out$df[4] # df
     
    index=index+1
}

Stats_adj_UKBB_le_con=Stats
Stats_adj_UKBB_le_con=data.frame(Stats_adj_UKBB_le_con)
colnames(Stats_adj_UKBB_le_con)=c('measure','estimate','p','t')
Stats_adj_UKBB_le_con$estimate=as.numeric(levels(Stats_adj_UKBB_le_con$estimate))[Stats_adj_UKBB_le_con$estimate]
Stats_adj_UKBB_le_con$p=as.numeric(levels(Stats_adj_UKBB_le_con$p))[Stats_adj_UKBB_le_con$p]
Stats_adj_UKBB_le_con$t=as.numeric(levels(Stats_adj_UKBB_le_con$t))[Stats_adj_UKBB_le_con$t]

In [ ]:
#Stats_adj_UKBB[,4][as.numeric(Stats_adj_UKBB[,3])>0.05]=NA
#Stats_adj[,4][as.numeric(Stats_adj[,4])>0]=NA 

plot_list.lh=list()
plot_list.rh=list()

for (i in 1:62) {
    if (i<32) {
        plot_list.lh[[(unlist(strsplit(as.character(Stats_adj_UKBB_le[i,1]),"_")))[4]]]=as.numeric(Stats_adj_UKBB_le[i,2]) }
    else {
        plot_list.rh[[(unlist(strsplit(as.character(Stats_adj_UKBB_le[i,1]),"_")))[4]]]=as.numeric(Stats_adj_UKBB_le[i,2])
    }
}

plot_list.lh[['bankssts']]=plot_list.lh[['middletemporal']]
plot_list.rh[['bankssts']]=plot_list.rh[['middletemporal']]

In [ ]:
atlas = 'aparc';
template_subject = 'fsaverage';
template_subjects_dir='/home/bic/fmorys/.local/share/fsbrain/subjects_dir/'
makecmap_options = list()
colFn_many_blues = colorRampPalette(RColorBrewer::brewer.pal(9, name="Blues"));
colFn_diverging= function(n) { grDevices::hcl.colors(n,palette='Blue-Red 3'); }
makecmap_options = list('colFn'=colFn_diverging, 'symm'=TRUE);


rgloptions = list("windowRect"=c(50, 50, 1000, 1000));
rglactions = list("snapshot_png"="/dagher/dagher11/filip/OBAD/figures/CT_ob_lean.png")


vis.region.values.on.subject(template_subjects_dir, template_subject, atlas, 
                             plot_list.lh, 
                             plot_list.rh,
                             draw_colorbar = TRUE,
                             bg='curv',
                             rgloptions=rgloptions,
                             makecmap_options=makecmap_options,
                             rglactions=rglactions);

## Calculate correlations between maps

In [ ]:
UKBB_est_le=Stats_adj_UKBB_le$estimate
UKBB_est_le_con=Stats_adj_UKBB_le_con$estimate

In [ ]:
ADNI_est_le=Stats_adj_ADNI_le$estimate
ADNI_est_le_con=Stats_adj_ADNI_le_con$estimate

In [ ]:
cor.test(UKBB_est_le,ADNI_est_le, method='spearman')

In [ ]:
cor.test(UKBB_est_le_con,ADNI_est_le_con, method='spearman')

# Run spin tests (https://github.com/frantisekvasa/rotate_parcellation)

In [ ]:
source("/dagher/dagher11/filip/Software/rotate_parcellation/R/perm.sphere.p.R")
source("/dagher/dagher11/filip/Software/rotate_parcellation/R/rotate.parcellation.R")

In [ ]:
Medioids=read.table('/dagher/dagher11/filip/OBAD/data/DKT31_vol_coords.csv', sep=',',header=T)
Cort_medioids=Medioids[35:96,]
Cort_medioids=select(Cort_medioids, medoid_x_mni, medoid_y_mni, medoid_z_mni)

### UKBB

In [ ]:
perm.id=rotate.parcellation(coord.l=as.matrix(Cort_medioids[c(1:31),]), coord.r=as.matrix(Cort_medioids[32:62,]), nrot=10000)
p_perm=perm.sphere.p(UKBB_est_con, ADNI_est_con, perm.id)
print(p_perm)
p_perm=perm.sphere.p(UKBB_est_le_con, ADNI_est_le_con, perm.id)
print(p_perm)
p_perm=perm.sphere.p(UKBB_est_ob_con, ADNI_est_ob_con, perm.id)
print(p_perm)



### ADNI

In [ ]:
#perm.id=rotate.parcellation(coord.l=as.matrix(Cort_medioids[c(1:31),]), coord.r=as.matrix(Cort_medioids[32:62,]), nrot=10000)
p_perm=perm.sphere.p(ADNI_est, UKBB_est, perm.id)
print(p_perm)
p_perm=perm.sphere.p(ADNI_est_le, UKBB_est_le, perm.id)
print(p_perm)
p_perm=perm.sphere.p(ADNI_est_ob, UKBB_est_ob, perm.id)
print(p_perm)


### Amyloid + tau

In [ ]:
perm.id=rotate.parcellation(coord.l=as.matrix(Cort_medioids[c(1:31),]), coord.r=as.matrix(Cort_medioids[32:62,]), nrot=10000)

p_perm=perm.sphere.p(ADNI_est, amyloid, perm.id)
print(p_perm)
p_perm=perm.sphere.p(UKBB_est, amyloid, perm.id)
print(p_perm)
p_perm=perm.sphere.p(ADNI_est, tau, perm.id)
print(p_perm)
p_perm=perm.sphere.p(UKBB_est, tau, perm.id)
print(p_perm)